# Generate Site Risk Scores
This notebook needs to be revised to support upcoming metadata changes (h3 ids in particular)

Also it is very messy. A one off notebook that will later serve as a basis for risk score generation

In [8]:
import geopandas as gpd
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import requests
import shapely
from tqdm.notebook import tqdm

In [9]:
endpoint = 'https://api.plastic.watch.earthrise.media/sites'
response = requests.get(endpoint, params={'limit':1500})
data = response.json()

In [10]:
gdf = gpd.GeoDataFrame.from_features(data['features'])
gdf['mean area'] = [None for _ in range(len(gdf))]
gdf['median area'] = [None for _ in range(len(gdf))]
gdf['max area'] = [None for _ in range(len(gdf))]
gdf['min area'] = [None for _ in range(len(gdf))]
gdf.head()

,geometry,area,clay content in soil (%),country,distance to waterway(m),drainage direction,elevation (m),fine earth density (kg / m^3),hydrologically adjusted elevations,id,...,soil great group,upstream drainage area (km^2),max area,mean area,median area,min area,population,population - 1km,population - 5km,risk
0,POINT (99.26969 8.93151),3430.234025443578,22.0,Thailand,218.39590537947285,northwest,14,128.0,3.8000001907348633,3765,...,0.0,0.033784687519073486,None,None,None,None,NaN,NaN,NaN,NaN
1,POINT (100.72792 16.33334),3214.4615579639003,28.0,Thailand,208.9648396943692,northwest,105,142.0,19.400001525878906,3769,...,1.0,0.008209395222365856,None,None,None,None,NaN,NaN,NaN,NaN
2,POINT (101.06602 16.24532),1223.2472461796515,30.0,Thailand,204.7642657416497,southwest,124,145.0,3.8000030517578125,3771,...,2.0,0.008213036693632603,None,None,None,None,NaN,NaN,NaN,NaN
3,POINT (100.87518 12.68301),19425.848410794126,31.0,Thailand,781.8894423776956,southeast,75,127.0,36.60000228881836,3774,...,0.0,0.15852290391921997,None,None,None,None,NaN,NaN,NaN,NaN
4,POINT (102.18610 12.65177),21392.74581407128,25.0,Thailand,0.0,northwest,19,123.0,0.0,3776,...,0.0,0.7843680381774902,None,None,None,None,NaN,NaN,NaN,NaN


In [11]:
for site_id in tqdm(gdf['id']):
    contours = requests.get(f"{endpoint}/{site_id}/contours").json()
    areas = [f['properties']['area (km^2)'] for f in contours['features']]
    if len(areas) > 0:
        areas = np.array(areas).astype(float)
        gdf.loc[gdf['id'] == site_id, 'mean area'] = np.mean(areas)
        gdf.loc[gdf['id'] == site_id, 'median area'] = np.median(areas)
        gdf.loc[gdf['id'] == site_id, 'max area'] = np.max(areas)
        gdf.loc[gdf['id'] == site_id, 'min area'] = np.min(areas)

KeyboardInterrupt: 

In [ ]:
k = 'upstream drainage area (km^2)'
gdf[k] = gdf[k].astype(float)
gdf.to_file('../data/site_metadata/full_v0_metadata.geojson', driver='GeoJSON')

In [ ]:
gdf['clay content in soil (%)'].hist()

In [ ]:
gdf['population'] = [None for _ in range(len(gdf))]

In [ ]:
gdf.loc[gdf['geometry'] == point, 'population'].values[0]

In [ ]:
gdf = gpd.read_file('../data/site_metadata/se_asia_metadata_2.0.geojson')
waterway_distance = np.array(gdf['waterway distance'])
waterway_distance[waterway_distance == -1] = 5000
gdf['waterway distance'] = waterway_distance
gdf.head()

In [ ]:
variable_classes = ['population', 'clay content in soil (%)', 'waterway distance', 'mean area']
stats = {}
for var in variable_classes:
    stats[var] = {}
    data = gdf[var].astype('float')
    gdf[var] = data
    if var == 'clay content in soil (%)':
        pass
    if var == 'waterway distance':
        data = np.log(data + 10)
    else:
        data = np.log(data)
    data.hist(bins=40)
    plt.title(var)
    plt.show()
    stdev = np.std(data)
    mean = np.mean(data)
    stats[var]['mean'] = mean
    stats[var]['std'] = stdev
stats

In [ ]:
import matplotlib.pyplot as plt
risk_score = []
for site_index in range(len(gdf)):
    site = gdf.iloc[site_index]
    site_deviation = {}
    for var in variable_classes:
        if var == 'clay content in soil (%)':
            data = site[var].astype('float')
        if var == 'waterway distance':
            data = site[var].astype('float')
            data = np.log(data + 10)
        else:
            data = np.log(site[var].astype('float'))
        #print(var, data)
        #print('average', var, stats[var]['mean'])
        #print('Deviation', (data - stats[var]['mean']) / stats[var]['std'])
        site_deviation[var] = (data - stats[var]['mean']) / stats[var]['std']
    risk = ((61 * site_deviation['mean area'] + 21 * site_deviation['population']) - (69 * site_deviation['waterway distance'] + 41 * site_deviation['clay content in soil (%)'])) / (69 + 61 + 41 + 21)
    risk_score.append(risk)
plt.hist(risk_score, bins=30)
plt.show()

In [ ]:
normed_risk = (np.array(risk_score) - np.nanmin(risk_score)) / (np.nanmax(risk_score) - np.nanmin(risk_score))
plt.hist(normed_risk, bins=35);

In [ ]:
new_gdf = gdf.copy()
new_gdf['risk'] = normed_risk
new_gdf.to_file('../data/site_metadata/se_asia_metadata_2.0.geojson', driver='GeoJSON')

In [ ]:
new_gdf

In [ ]:
from tensorflow import keras
model = keras.models.load_model('../models/spectrogram_v0.0.11.1_2021-11-14.h5')

In [ ]:
model.layers[0].output_shape

In [ ]:
request = requests.get('https://www.ncdc.noaa.gov/cdo-web/api/v2/datasets', params={'limit':1000, 'Token': 'LKmcuTZgdJdoXFCyfcTBwkonjAlYPMWc'})
request.json()

In [ ]:
from scripts import dl_utils
gdf = gpd.read_file('../data/site_metadata/se_asia_metadata_risk.geojson')

In [ ]:
pop_name = 'population - 10km'
gdf[pop_name] = [None for _ in range(len(gdf))]
for point in tqdm(gdf['geometry'][:2]):
  if gdf.loc[gdf['geometry'] == point, pop_name].values[0] == None:
    bbox = [[list(coords) for coords in dl_utils.rect_from_point([point.x, point.y], 0.09)['coordinates'][0]]]
    geojson = {"type":"FeatureCollection",
              "features":
                    [{"type":"Feature",
                      "properties":{},
                      "geometry":
                        {"type":
                        "Polygon",
                        "coordinates": bbox
                        }
                    }
                    ]
            }
    geojson_str = str(geojson).replace("\'", '\"')
    try:
      r = requests.get(f'https://api.worldpop.org/v1/services/stats?dataset=wpgppop&year=2020&geojson={geojson_str}&runasync=false')
      pop = r.json()['data']['total_population']
      gdf.loc[gdf['geometry'] == point, pop_name] = pop
    except Exception as e:
      print("failed", point)
      print(e)


In [ ]:
gdf['population - 5km'] = gdf['population']

In [ ]:
import json
json.dumps(geojson)

In [ ]:
gdf

In [ ]:
gdf.to_file('../data/site_metadata/full_v0_metadata.geojson', driver='GeoJSON')

In [ ]:
pop_task = requests.get(f'https://api.worldpop.org/v1/tasks/{r.json()["taskid"]}')
pop_task.json()

In [ ]:
pop_task = requests.get(f'https://api.worldpop.org/v1/tasks/{r.json()["taskid"]}')
pop_task.json()

In [ ]:
from scripts import dl_utils
shapely.geometry.Polygon(dl_utils.rect_from_point([gdf['geometry'][0].x, gdf['geometry'][0].y], 0.001)['coordinates'][0])

In [ ]:
plt.scatter(gdf['distance to waterway(m)'], gdf['mean area'], s=1)
plt.ylabel('Mean Area')
plt.xlabel('Distance to Waterway')
plt.show()

In [ ]:
id_val = gdf[gdf['name'] == gdf['name'][0]]['id'].values[0]
contours = requests.get(f"{endpoint}/{id_val}/contours").json()

In [ ]:
for f in contours['features']:
    print(f['properties']['area (km^2)'])

In [ ]:
input_file = 'indonesia_v0_mask_window_8_upsampled_4_contours_model_v0.0.11_ensemble-8-25-21'
contours = gpd.read_file(f'../data/model_outputs/site_contours/v2.1/{input_file}.geojson')

In [ ]:
site_names = contours['name'].unique()
#contours.loc[contours['area (km^2)'] < 0.005, 'area (km^2)'] = None

In [ ]:
metadata = pd.read_csv('../data/site_metadata/SE_ASIA_METADATA.csv')
metadata_gdf = gpd.GeoDataFrame(geometry = [shapely.geometry.Point([lon, lat]) for lon, lat in zip(metadata['lon'], metadata['lat'])])
for col in metadata.columns[4:]:
    metadata_gdf[col] = metadata[col]
metadata_gdf.head(5)


In [ ]:
median_area = contours.groupby('name').median()['area (km^2)'] * 1000000

In [ ]:
metadata_gdf['name'] = '' * len(metadata_gdf)
metadata_gdf.head(5)

In [ ]:
count = 0
for name in metadata_gdf['name']:
    if len(name) > 0:
        count += 1
        print(name)
print(count)

In [ ]:
for point in tqdm(metadata_gdf['geometry']):
    for contour, name in zip(contours['geometry'], contours['name']):
        if contour is not None:
            if contour.buffer(0.01).contains(point):
                metadata_gdf.loc[metadata_gdf['geometry'] == point, 'name'] = name
            


In [ ]:
count = 0
for elem in contours.groupby('name').count()['geometry']:
    if elem > 0:
        count+=1
print(count)

In [ ]:
for data_site in contours['geometry']:
    pass

In [ ]:
data_site

In [ ]:
distance = []
for data_site in contours['geometry']:
    distance.append(np.mean(metadata_gdf[[site.buffer(0.001).contains(data_site) for site in metadata_gdf['geometry']]])['distance to waterway(m)'])

In [ ]:
contours.groupby('name').mean().sort_values(by='area (km^2)', ascending=False).head(20)

In [ ]:
biggest_sites = contours.groupby('name').mean().sort_values(by='area (km^2)', ascending=False).index.values

In [ ]:
change = []
for n in site_names:
    if np.isfinite(np.nanmean(contours[contours['name'] == n]['area (km^2)'])):
        rolling_vals = contours[contours['name'] == n]['area (km^2)'].rolling(3).mean()[np.isfinite(contours[contours['name'] == n]['area (km^2)'].rolling(3).mean())]
        if len(rolling_vals) > 1:
            try:
                slope, residual = np.polyfit(range(len(rolling_vals)), rolling_vals, 1)
                change.append(slope)
                #plt.plot(range(len(rolling_vals)), rolling_vals, '-o')
                #plt.title(f'{slope:.4f}, {residual:.4f}')
                #plt.show()

            except Exception as e:
                print(e)
                print(len(rolling_vals))
        else:
            change.append(np.nan)
    else:
        change.append(np.nan)

In [ ]:
import pandas as pd
change_df = pd.DataFrame({'name': site_names, 'change': change})
change_df[np.isfinite(change_df['change']) == True].sort_values(by='change', ascending='True').head(30)


In [ ]:
for g in contours[contours['name'] == 'java_v0.0.7_2019-01-01_2021-06-01mosaic-median_71']['geometry']:
    display(g)

In [ ]:
rolling_vals = contours[contours['name'] == n]['area (km^2)'].rolling(3).mean()[np.isfinite(contours[contours['name'] == n]['area (km^2)'].rolling(3).mean())]
np.polyfit(range(len(rolling_vals)), rolling_vals, 1)[0]

In [ ]:
for n in site_names[:10]:
    if np.isfinite(np.nanmean(contours[contours['name'] == n]['area (km^2)'])):
        rolling_vals = contours[contours['name'] == n]['area (km^2)'].rolling(3).mean()[np.isfinite(contours[contours['name'] == n]['area (km^2)'].rolling(3).mean())]
        if len(rolling_vals > 2):
            contours[contours['name'] == n]['area (km^2)'].rolling(16).mean().plot()
            plt.title('trend')
            plt.show()
            contours[contours['name'] == n]['area (km^2)'].rolling(3).mean().plot()
            plt.show()